# Huấn luyện AI Gomoku với Tính Năng Nâng Cao

Notebook đã được cập nhật với các tính năng mới giúp AI học hiệu quả hơn:
- **Curriculum Learning**: Học từ đơn giản đến phức tạp
- **Opponent Pool**: Đa dạng hóa đối thủ trong huấn luyện
- **Phần thưởng cải tiến**: Khuyến khích thắng nhanh, tránh hòa

## 1. Thiết lập môi trường Colab

In [ ]:
# Kiểm tra GPU
!nvidia-smi

# Cài đặt các thư viện cần thiết
!pip install pygame numpy torch torchvision tensorboard matplotlib pandas tqdm

In [ ]:
# Kết nối Google Drive để lưu trữ mô hình
from google.colab import drive
drive.mount('/content/drive')

# Tạo thư mục dự án
import os
DRIVE_PROJECT_DIR = '/content/drive/MyDrive/BotGomuko'
DRIVE_MODELS_DIR = f"{DRIVE_PROJECT_DIR}/saved_models"
DRIVE_LOGS_DIR = f"{DRIVE_PROJECT_DIR}/logs"

# Tạo cấu trúc thư mục
for directory in [DRIVE_PROJECT_DIR, DRIVE_MODELS_DIR, DRIVE_LOGS_DIR]:
    os.makedirs(directory, exist_ok=True)
    
print(f"Google Drive đã kết nối tại {DRIVE_PROJECT_DIR}")

## 2. Clone Repository từ GitHub

Tải mã nguồn trực tiếp từ GitHub repository chính thức:

In [ ]:
# Clone repository từ GitHub
!git clone https://github.com/phattran1023/ML-gomuko.git

# Di chuyển vào thư mục dự án
%cd ML-gomuko

# Kiểm tra cấu trúc thư mục
!ls -la

In [ ]:
# Thiết lập pygame cho Colab (headless mode)
import os
os.environ['SDL_VIDEODRIVER'] = 'dummy'

# Khởi tạo pygame
import pygame
pygame.init()
pygame.display.set_mode((1, 1))
print("Pygame đã sẵn sàng trong chế độ headless")

## 3. Tạo Script Tự Động Lưu vào Drive

Script này đảm bảo mô hình được lưu vào Drive và không bị mất khi Colab ngắt kết nối:

In [ ]:
%%writefile save_to_drive.py
import os
import sys
import time
import shutil
from pathlib import Path

def create_drive_model_path(model_path, drive_base_dir):
    """Tạo đường dẫn tương ứng trên Google Drive"""
    path = Path(model_path)
    drive_path = os.path.join(drive_base_dir, path.name)
    return drive_path

def setup_drive_auto_save(original_args, drive_models_dir, drive_logs_dir):
    """Thiết lập tự động lưu mô hình vào Google Drive"""
    import sys
    import time
    from main import self_train

    # Áp dụng tối ưu hóa cho Colab
    try:
        from colab_optimizer import optimize_for_colab, patch_agent_for_colab
        optimize_for_colab()
        # Áp dụng patch cho RLAgent
        from agents.rl_agent import RLAgent
        patch_agent_for_colab(RLAgent)
        print("Đã áp dụng tối ưu hóa cho Colab")
    except Exception as e:
        print(f"Không thể áp dụng tối ưu hóa Colab: {e}")

    # Phân tích tham số
    args = {}
    i = 0
    while i < len(original_args):
        if original_args[i].startswith('--'):
            param = original_args[i][2:]
            if i + 1 < len(original_args) and not original_args[i+1].startswith('--'):
                args[param] = original_args[i+1]
                i += 2
            else:
                args[param] = True
                i += 1
        else:
            i += 1

    # Thiết lập tham số
    total_episodes = int(args.get('episodes', 10000))
    save_interval = int(args.get('save-interval', 100))
    resume = 'resume' in args
    use_curriculum = 'curriculum' in args
    use_opponent_pool = not ('no-opponent-pool' in args)
    
    # Đường dẫn mô hình
    model_path = args.get('model', 'saved_models/rl_self_train.pt')
    
    # Tạo phiên bản drive của model_path
    drive_model_path = create_drive_model_path(model_path, drive_models_dir)
    
    # Tạo thư mục TensorBoard trên Drive
    timestr = time.strftime('%Y%m%d-%H%M%S')
    drive_log_dir = f"{drive_logs_dir}/self_train_{timestr}"
    os.makedirs(drive_log_dir, exist_ok=True)
    
    print(f"\n=== AUTO-SAVE TO GOOGLE DRIVE ENABLED ===")
    print(f"Model path: {drive_model_path}")
    print(f"TensorBoard logs: {drive_log_dir}")
    
    # Nếu resume và có model trong Drive, sao chép về Colab
    if resume and os.path.exists(drive_model_path):
        print(f"Tìm thấy mô hình trên Drive, sao chép về Colab để tiếp tục huấn luyện...")
        os.makedirs(os.path.dirname(model_path), exist_ok=True)
        shutil.copy2(drive_model_path, model_path)
    
    # Hook for model saving
    original_save = self_train.__globals__['RLAgent'].save
    
    def save_hook(self, filepath):
        # Gọi hàm save gốc
        original_save(self, filepath)
        
        # Nếu là checkpoint hoặc mô hình cuối cùng, lưu vào Drive
        if '_ep' in filepath or 'self_train.pt' in filepath:
            drive_path = create_drive_model_path(filepath, drive_models_dir)
            os.makedirs(os.path.dirname(drive_path), exist_ok=True)
            shutil.copy2(filepath, drive_path)
            print(f"Đã sao chép mô hình vào Drive: {drive_path}")
            
            # Sao chép các biểu đồ liên quan
            for suffix in ['_progress_final.png', '_opponents_ep', '_curriculum_ep', '_win_rates_ep']:
                base_name = os.path.splitext(filepath)[0]
                for f in os.listdir(os.path.dirname(filepath)):
                    if f.startswith(os.path.basename(base_name)) and suffix in f:
                        source_path = os.path.join(os.path.dirname(filepath), f)
                        dest_path = os.path.join(drive_models_dir, f)
                        shutil.copy2(source_path, dest_path)
                        print(f"Đã sao chép biểu đồ vào Drive: {dest_path}")
    
    # Thay thế hàm save
    self_train.__globals__['RLAgent'].save = save_hook
    
    # Start training với các tính năng mới
    self_train(
        total_episodes=total_episodes,
        save_interval=save_interval,
        model_path=model_path,
        resume=resume,
        use_tensorboard=True,
        use_curriculum=use_curriculum,
        use_pool=use_opponent_pool,
        verbose=1
    )
    
if __name__ == "__main__":
    if len(sys.argv) < 3:
        print("Usage: python save_to_drive.py <drive_models_dir> <drive_logs_dir> [training args...]")
        sys.exit(1)
        
    drive_models_dir = sys.argv[1]
    drive_logs_dir = sys.argv[2]
    original_args = sys.argv[3:]
    
    setup_drive_auto_save(original_args, drive_models_dir, drive_logs_dir)

## 4. Kiểm tra và Cập Nhật Repository (Tùy chọn)

Kiểm tra và cập nhật phiên bản mới nhất từ GitHub nếu cần:

In [ ]:
# Kiểm tra phiên bản hiện tại
!git log -1 --pretty=format:"%h - %an, %ar : %s"

# Cập nhật repository nếu cần
!git pull origin main

## 5. Huấn luyện với Tính Năng Nâng Cao

Chọn một trong các phương pháp huấn luyện sau:

### Giải thích các tính năng nâng cao:

#### 1. Curriculum Learning
- **Giai đoạn 1**: Bàn cờ trống, học từ đầu
- **Giai đoạn 2**: Bắt đầu với 3-5 quân cờ ngẫu nhiên
- **Giai đoạn 3**: Bắt đầu với 6-10 quân cờ
- **Giai đoạn 4**: Tình huống tấn công (3 quân liên tiếp)
- **Giai đoạn 5**: Tình huống phòng thủ (đối thủ có 3 quân liên tiếp)

#### 2. Opponent Pool
- Lưu 5 phiên bản gần nhất của AI làm đối thủ
- Cứ 10 tập sẽ đấu với một trong các phiên bản cũ
- Giúp AI đa dạng hóa chiến thuật

#### 3. Phần thưởng cải tiến
- Phần thưởng âm cho kết quả hòa (-0.1)
- Phần thưởng thêm cho thắng nhanh (tối đa +1.0)

In [ ]:
# ===== KHUYẾN NGHỊ: Huấn luyện đầy đủ với tất cả tính năng =====
!python save_to_drive.py "$DRIVE_MODELS_DIR" "$DRIVE_LOGS_DIR" --episodes 10000 --save-interval 500 --curriculum

In [ ]:
# Huấn luyện với chỉ Curriculum Learning
!python save_to_drive.py "$DRIVE_MODELS_DIR" "$DRIVE_LOGS_DIR" --episodes 5000 --save-interval 200 --curriculum --no-opponent-pool

In [ ]:
# Tiếp tục huấn luyện từ checkpoint
!python save_to_drive.py "$DRIVE_MODELS_DIR" "$DRIVE_LOGS_DIR" --episodes 5000 --save-interval 200 --curriculum --resume

## 6. Giám sát Quá trình Huấn luyện

Theo dõi quá trình huấn luyện bằng TensorBoard:

In [ ]:
# Khởi động TensorBoard để theo dõi quá trình huấn luyện
%load_ext tensorboard
%tensorboard --logdir logs

## 7. Đánh giá Mô hình

Đánh giá mô hình bằng script đánh giá mới:

In [ ]:
# Sao chép mô hình từ Drive để đánh giá
!mkdir -p saved_models
!cp "$DRIVE_MODELS_DIR/rl_self_train.pt" saved_models/ 2>/dev/null || echo "Không tìm thấy mô hình, hãy huấn luyện trước"

# Đánh giá toàn diện
!python evaluate_model.py --model saved_models/rl_self_train.pt --mode all --num-games 50

# Hiển thị kết quả đánh giá
from IPython.display import Image
Image(filename='model_evaluation.png')

In [ ]:
# Đánh giá với MCTS (mạnh hơn nhưng chậm hơn)
!python evaluate_model.py --model saved_models/rl_self_train.pt --mode random --num-games 10 --use-mcts --simulations 50

In [ ]:
# So sánh các phiên bản (checkpoint) khác nhau
!mkdir -p saved_models
!cp "$DRIVE_MODELS_DIR"/*_ep*.pt saved_models/ 2>/dev/null || echo "Không tìm thấy checkpoint"
!python evaluate_model.py --model saved_models/rl_self_train.pt --mode compare --num-games 10 --models-dir saved_models

## 8. Hiển thị Biểu đồ Huấn luyện

Xem các biểu đồ phân tích quá trình huấn luyện:

In [ ]:
# Hiển thị biểu đồ tiến trình huấn luyện từ Drive
import glob
from IPython.display import Image, display

# Kiểm tra biểu đồ tiến trình
progress_plots = sorted(glob.glob(f"{DRIVE_MODELS_DIR}/*progress*.png"))
if progress_plots:
    print(f"Hiển thị biểu đồ tiến trình mới nhất: {progress_plots[-1]}")
    display(Image(filename=progress_plots[-1]))
else:
    print("Không tìm thấy biểu đồ tiến trình")

# Kiểm tra biểu đồ curriculum
curriculum_plots = sorted(glob.glob(f"{DRIVE_MODELS_DIR}/*curriculum*.png"))
if curriculum_plots:
    print(f"\nHiển thị biểu đồ curriculum learning mới nhất: {curriculum_plots[-1]}")
    display(Image(filename=curriculum_plots[-1]))

# Kiểm tra biểu đồ opponent distribution
opponent_plots = sorted(glob.glob(f"{DRIVE_MODELS_DIR}/*opponents*.png"))
if opponent_plots:
    print(f"\nHiển thị biểu đồ phân phối đối thủ mới nhất: {opponent_plots[-1]}")
    display(Image(filename=opponent_plots[-1]))